# Setup the functional API

In [1]:
from config import config
from src.llm.openrouter import get_openrouter
from langgraph.func import task, entrypoint
max_tokens = 1024*8
llm = get_openrouter(max_tokens=max_tokens)



In [ ]:
# setup tasks for generating 3 example code snippets


@task
def generate_natural_language_code_snippet(error_message: str):
    prompt = f"""
    Generate a natural language code snippet that will help me fix the following error message. It should be in simple english and not technical jargon.
    it should include all the input/output parameters and the expected output.
    {error_message}
    """
    result = llm.invoke(prompt)
    content = result.get("content", "")  # Safely access the "content" key
    return content




@task
def generate_code_snippet(natural_language_code_snippet: str):
    prompt = f"""
    Generate a code snippet that will help me fix the following error message. It should be in python.
    {natural_language_code_snippet}
    """
    result = llm.invoke(prompt)
    content = result.get("content", "")  # Safely access the "content" key
    return content



In [ ]:
@task
def generate_plan_to_implement_the_code_prompt(prompt: str):
    prompt = f"""
    Generate a plan to implement the following code snippet.
    {prompt}
    """
    return llm.invoke(prompt).content



@entrypoint
def _start_workflow(state:dict, input_data: dict):
    input_text = input_data.get("input_text", "")
    
    
    error_message = input_data.get("error_message", "")
    natural_language_code_snippet = generate_natural_language_code_snippet(error_message)
    code_snippet = generate_code_snippet(natural_language_code_snippet)
    plan_to_implement_the_code_prompt = generate_plan_to_implement_the_code_prompt(code_snippet)
    return plan_to_implement_the_code_prompt



